In [1]:
using Pkg
Pkg.add("XLSX")
import XLSX

#Load Oscar
using Oscar

    Updating registry at `~/.julia/registries/General`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.6/Project.toml`
  No Changes to `~/.julia/environments/v1.6/Manifest.toml`


 -----    -----    -----      -      -----   
|     |  |     |  |     |    | |    |     |  
|     |  |        |         |   |   |     |  
|     |   -----   |        |     |  |-----   
|     |        |  |        |-----|  |   |    
|     |  |     |  |     |  |     |  |    |   
 -----    -----    -----   -     -  -     -  

...combining (and extending) ANTIC, GAP, Polymake and Singular
Version 0.8.2 ... 
 ... which comes with absolutely no warranty whatsoever
Type: '?Oscar' for more information
(c) 2019-2022 by The Oscar Development Team


<font size="5">
In the first step the variables and the corresponding polynomial ring are defined. The coefficients of the polynomials can either be zero or one only (from the 2-elements field). 
</font>

In [2]:
R1 = ResidueRing(ZZ, 2);
vo= ["e", "h", "I", "a", "c", "p", "H", "d", "x"];
myR, (e, h, I, a, c, p, H, d, x) = PolynomialRing(R1, vo);
myvars=[e, h, I, a, c, p, H, d, x];

<font size="5">
Now your xlsx-table is loaded. The table consists of entries 0 and 1.
The array 'indices' is also loaded. It includes an own labeling of the objects in the table (if wanted). 
</font>

In [3]:
chartable=XLSX.readdata("../Data/Retirement_short.xlsx", "Tabelle1", "A3:I1526");
indices=1:1524;

<font size="5">
The analysis can be restricted to a subset of the characteristics. Select some of them.
</font>

In [4]:
varnum=9; choice=1:9

1:9

<font size="5">
According to your choice, the Boolean entries are transformed into an array of polynomials.
</font>

In [5]:
chartable=chartable[:,choice];
myvars=myvars[choice];
expr_list=prod(myvars+ myvars.^0 -chartable[1,:]); 
for i=2:size(chartable,1)
    expr_list=vcat(expr_list, prod(myvars+ myvars.^0 -chartable[i,:])); 
end

<font size="5">
For the weighting of the monomials, we do some statistics about the marginal distribution of characteristics
</font>

In [6]:
monweight=ones(Int,varnum);
for i=1:size(myvars,1)
    print(myvars[i]);
    print(" true=");
    print(sum(chartable[:,i]));
    print(" false=");
    print(size(chartable,1)-sum(chartable[:,i]));
    print("\n");
    monweight[i]=sum(chartable[:,i])*(size(chartable,1)-sum(chartable[:,i]));
end
monweight

e true=666 false=858
h true=66 false=1458
I true=255 false=1269
a true=411 false=1113
c true=240 false=1284
p true=312 false=1212
H true=727 false=797
d true=374 false=1150
x true=255 false=1269


9-element Vector{Int64}:
 571428
  96228
 323595
 457443
 308160
 378144
 579419
 430100
 323595

<font size="5">
The algorithm generates an ideal in the set of Boolean Polynomials. This ideal represents every combination of characteristics which is not to be found in your table. The Gröbner Basis of this ideal is computed.
</font>

In [7]:
uq_expr_list=unique(expr_list);
expr=sum(uq_expr_list)+1;
generator=myvars.^2+myvars;
Y, m = quo(myR, ideal(myR, vcat(generator, expr)));
GB = groebner_basis(ideal(myR, ordering = vcat(generator, expr)), wdeglex(gens(myR),monweight))

LoadError: MethodError: no method matching ideal(::NmodMPolyRing; ordering=nmod_mpoly[e^2 + e, h^2 + h, I^2 + I, a^2 + a, c^2 + c, p^2 + p, H^2 + H, d^2 + d, x^2 + x, e*h*I*a*c*p*H + e*h*I*a*c*p*d*x + e*h*I*a*c*p*d + e*h*I*a*c*H*d*x + e*h*I*a*c*H*d + e*h*I*a*c*H*x + e*h*I*a*c*d*x + e*h*I*a*c*d + e*h*I*a*c + e*h*I*a*p*H*d + e*h*I*a*p*H*x + e*h*I*a*p*x + e*h*I*a*p + e*h*I*a*H*d + e*h*I*a*H*x + e*h*I*a*H + e*h*I*c*p*H + e*h*I*c*p*d + e*h*I*c*p*x + e*h*I*c*p + e*h*I*c*H*d*x + e*h*I*c*H*x + e*h*I*c*d*x + e*h*I*c*d + e*h*I*c*x + e*h*I*c + e*h*I*p*H*d*x + e*h*I*p*H*x + e*h*I*p*H + e*h*I*H*x + e*h*I*x + e*h*a*c*p*H*d*x + e*h*a*c*p*H*d + e*h*a*c*p*d*x + e*h*a*c*p*d + e*h*a*c*p*x + e*h*a*c*H*d*x + e*h*a*c*d*x + e*h*a*c*d + e*h*a*c*x + e*h*a*p*H*d*x + e*h*a*p*H*x + e*h*a*p*d + e*h*a*p*x + e*h*a*p + e*h*a*H*d + e*h*a*H*x + e*h*a*H + e*h*a*d*x + e*h*a*x + e*h*c*p*H*x + e*h*c*p*H + e*h*c*p*d*x + e*h*c*p*x + e*h*c*H*d*x + e*h*c*H*d + e*h*c*H*x + e*h*c*d*x + e*h*c*d + e*h*p*H*d*x + e*h*p*d + e*h*p*x +

<font size="5">
According to the "rules" included in the Gröbner Basis the simplification can be applied to all entries in the loaded table:
</font>

In [8]:
wrtnorm=0;
for i = 1:size(uq_expr_list,1)
    #res=m(uq_expr_list[i]); simplify!(res);
    res=normal_form(uq_expr_list[i],GB);
    #res=m(normal_form(uq_expr_list[i],GB)); simplify!(res);
    counter=0;
    for j = 1:size(expr_list,1)
        if (uq_expr_list[i]==expr_list[j])
            counter=counter+1;
            printstyled(indices[j], color=:red);
            print(" ");
        end
    end
    printstyled(" (#", color=:red);
    printstyled(counter, color=:red);
    printstyled(")", color=:red);
    print("\n");
    printstyled(factor(uq_expr_list[i]), color=:green);
    if (wrtnorm==1)
        printstyled("=\n", color=:green);
        printstyled(uq_expr_list[i], color=:green);
    end
    print("\n");
    printstyled(res, bold=:true);
    print("\n\n");
end

LoadError: UndefVarError: GB not defined